In [ ]:
import pandas as pd
import numpy as np

from funcs.ad import aggregate_pvalues, get_sigma_values
from funcs.spirelations import (b_from_lx_reiners,
                                p_spi_lanza12,
                                wrap_obstimes,
                                calculate_relative_velocity)

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from adjustText import adjust_text

from astropy import units as u
from astropy.constants import R_sun, R_jup

import seaborn as sns


def colorcode_rotation(st_rotp):
    """Define color code and symbol marker for rotation period.

    Parameters
    ----------
    st_rotp : float
        Rotation period in days.

    Returns
    -------
    color : str
        Color code.
    marker : str
        Symbol marker.
    """
    if st_rotp < 10.:
        color = "green"
        symbol = "o"

    elif (st_rotp >= 10.) & (st_rotp < 15.):
        color = "blue"
        symbol = "x"

    elif (st_rotp >= 15.):
        color = "black"
        symbol = "d"

    else:
        color = "lightgrey"
        symbol = "s"

    return color, symbol


# define the colors and symbols for the different rotation periods
mean_std["color"], mean_std["symbol"] = zip(
    *mean_std.st_rotp.apply(colorcode_rotation))


In [1]:
import pandas as pd
from funcs.ad import aggregate_pvalues
from funcs.spirelations import b_from_lx_reiners, wrap_obstimes, p_spi_lanza12

# read in AD test results
adtests = pd.read_csv("../results/multistar_adtest.csv")

# read in the table of known parameters from the NASA Exoplanet Archive and TESS
sps_w_ad = pd.read_csv(
    "../results/params_of_star_planet_systems_with_AD_tests.csv")

# rename column tic_id to TIC
sps_w_ad.rename(columns={"tic_id": "TIC"}, inplace=True)

# read in age and rotation period from the literature search
literature_params = pd.read_csv(
    "../results/2022_08_stellar_params.csv")

# aggregate the p-values
mean_std = aggregate_pvalues(adtests, subsample="ED>1s", period="orbit")

# merge the AD test results with the stellar parameters
mean_std = mean_std.merge(literature_params, on=["TIC"], how="left")

# add semi-major axis in AU to the table
mean_std = mean_std.merge(sps_w_ad[["TIC", "pl_orbsmax", "st_rad_kepler",
                                    "st_rad_tess", "pl_radj", "pl_orbper_kepler",
                                    "pl_orbper_tess"]],
                            on="TIC", how="left")

# rename columns and fill NaNs will TESS values
mean_std = mean_std.rename(columns = {"pl_orbsmax": "a_au",})
mean_std["st_rad"] =  mean_std.st_rad_kepler.fillna(mean_std.st_rad_tess)
mean_std["orbper_d"] =  mean_std.pl_orbper_kepler.fillna(mean_std.pl_orbper_tess)

# calculate B field from X-ray luminosity and stellar radius
mean_std["B_G"] = mean_std.apply(lambda x: b_from_lx_reiners(x.xray_flux_erg_s, x.st_rad ), axis=1)


In [ ]:


# get the flare table
flares = pd.read_csv("../results/2022_07_flares_vetted.csv")

# get the observing time for each star using the flare tables
mean_std["obstime_d"] = mean_std.apply(lambda x: wrap_obstimes(str(x.TIC), 
                                                 flares), axis=1)

# calculate the number of covered orbits
mean_std["orbits_covered"] = mean_std["obstime_d"] / mean_std["orbper_d"]

# calculate relative velocity between stellar rotation at the planetary orbit
# and the orbital velocity of the planet in km/s
mean_std["v_rel_km_s"] = mean_std.apply(lambda x: 
                                        calculate_relative_velocity(x.a_au,
                                        x.orbper_d, x.st_rotp), axis=1)

(2 * np.pi * mean_std.a_au *
                                (1/mean_std.orbper_d - 1/mean_std.st_rotp) /
                                 24. / 3600. * 149597870.700)

# calculate the SPI power from the Lanza 2012 scaling relation
mean_std["p_spi_erg_s"] = mean_std.apply(lambda x: p_spi_lanza12(np.abs(x.v_rel_km_s),
                                                           x.B_G, x.pl_radj), axis=1)                    

In [ ]:
# ------------------------------------------------------------
# Plotting setup
# get reference sigma values
sigmas, sigma_labels = get_sigma_values()

# define legend elements
legend = [Line2D([0], [0], marker='X', color='w',
          markerfacecolor='blue', markersize=10),
          Line2D([0], [0], marker='d', color='w',
          markerfacecolor='black', markersize=10),
          Line2D([0], [0], marker='o', color='w',
          markerfacecolor='green', markersize=10),
          Line2D([0], [0], marker='s', color='w',
          markerfacecolor='lightgrey', markersize=10)]

# define legend labels
labels = [r"$10 \leq P_{rot} < 15$ d",
          r"$P_{rot} \geq 15$ d",
          r"$P_{rot} < 10$ d",
          r"no $P_{rot}$"]

# ------------------------------------------------------------

In [ ]:
# P-VALUE VS. SEMI-MAJOR AXIS

# set up figure
fig, ax = plt.subplots(figsize=(10, 6.5))

data = mean_std[mean_std.p_spi.notnull()]

texts = []

# plot the data
for color, group in data.groupby("color"):
    plt.scatter(group["p_spi"], group["mean"], marker=group["symbol"].iloc[0],
                c=color, s=50., alpha=1.)
    for star, row in group.iterrows():
        texts.append(plt.text(x=row["p_spi"], y=row["mean"], s=row["ID"],
                    fontsize=11, ha="right", va="top", rotation=0))
    # if (color != "lightgrey") & (color != "blue"):
    #     sns.kdeplot(data=group, x="p_spi", y="mean", fill=True,
    #                 color=color, common_norm=True, alpha=.4, log_scale=True,
    #                 cut=0., levels=[0.32, 0.68, 0.95, 0.997])


# log scale
plt.yscale("log")
plt.xscale("log")

# # add sigma lines
for sigma, label in list(zip(sigmas, sigma_labels)):
    plt.axhline(sigma, color="grey", linestyle="-.")
    plt.text(9e7, sigma, label, fontsize=15,
             color="k", verticalalignment="center")

# labels
plt.ylabel("p-value of AD test")
plt.xlabel("P$_{SPI}$ [some unit]")

# legend
plt.legend(legend[1:3], labels[1:3], loc=(0, 0.08), fontsize=15)

# limits
plt.xlim(5e2, 8e7)
plt.ylim(1.5e-3, 1)

adjust_text(texts, arrowprops=dict(arrowstyle="-", color='k', lw=0.5))
# save figure
plt.savefig("../results/plots/2022_11_22_adtests_vs_pspi_kde.png",
            dpi=300)


# ------------------------------------------------------------


In [ ]:
# P-VALUE VS. SEMI-MAJOR AXIS

# set up figure
fig, ax = plt.subplots(figsize=(10, 6.5))


# plot the data
for color, group in mean_std.groupby("color"):
    plt.scatter(group["a_au"], group["mean"], marker=group["symbol"].iloc[0],
                c=color, s=90., alpha=1.)
    if (color != "lightgrey") & (color != "blue"):
        sns.kdeplot(data=group, x="a_au", y="mean", fill=True,
                    color=color, common_norm=True, alpha=.4, log_scale=True,
                    cut=0., levels=[0.32, 0.68, 0.95, 0.997])


# log scale
plt.yscale("log")
plt.xscale("log")

# add sigma lines
for sigma, label in list(zip(sigmas, sigma_labels)):
    plt.axhline(sigma, color="grey", linestyle="-.")
    plt.text(.3, sigma, label, fontsize=15,
             color="k", verticalalignment="center")

# labels
plt.ylabel("p-value of AD test")
plt.xlabel("semi-major axis (AU)")

# legend
plt.legend(legend, labels, loc=(0, 0.08), fontsize=15)

# save figure
plt.savefig("../results/plots/2022_11_22_adtests_vs_semimajor_axis_kde.png",
            dpi=300)

# ------------------------------------------------------------


In [ ]:
# P-VALUE VS. SEMI-MAJOR AXIS

# set up figure
fig, ax = plt.subplots(figsize=(10, 6.5))

mean_std["vrellog"] = np.sign(mean_std.v_rel_km_s) * np.log10(
                                                np.abs(mean_std.v_rel_km_s))
# plot the data
for color, group in mean_std.groupby("color"):
    plt.scatter(group["vrellog"], group["mean"], marker=group["symbol"].iloc[0],
                c=color, s=90., alpha=1.)
    if (color != "lightgrey") & (color != "blue"):
        sns.kdeplot(data=group, x="vrellog", y="mean", fill=True,
                    color=color, common_norm=True, alpha=.4, #log_scale=True,
                    cut=0., levels=[0.32, 0.68, 0.95, 0.997])


# log scale
plt.yscale("log")
# plt.xscale("log")

# # add sigma lines
# for sigma, label in list(zip(sigmas, sigma_labels)):
#     plt.axhline(sigma, color="grey", linestyle="-.")
#     plt.text(1900, sigma, label, fontsize=15,
#              color="k", verticalalignment="center")

# labels
plt.ylabel("p-value of AD test")
plt.xlabel("log relative velocity [km/s]")

# limits
# plt.xlim(3, 1800)
plt.ylim(1.5e-3, 1)

# legend
plt.legend(legend, labels, loc=(.7, 0.08), fontsize=15)

# save figure
plt.savefig("../results/plots/2022_11_22_adtests_vs_vrel_kde.png",
            dpi=300)

# ------------------------------------------------------------


In [ ]:
minper, maxper = mean_std.orbper_d.min(), mean_std.orbper_d.max()

# set up figure
fig, ax = plt.subplots(figsize=(7, 5.5))

bins = np.logspace(np.log10(minper), np.log10(maxper), 20)

for color, group in mean_std.groupby("color"):
    # plt.hist(group.orbper_d, color=color, bins=bins)
    if color != "lightgrey":
        sns.kdeplot(data=group, x="orbper_d",
                    fill=True, alpha=0.4, color=color)

plt.xscale("log")
# plt.yscale("log")
plt.xlim(minper, maxper)
# plt.ylim(minper, maxper)
# plt.xlabel("rotation period [days]")
plt.xlabel("orbital period [days]")


In [ ]:
minflares, maxflares = mean_std.number_of_flares.min(), mean_std.number_of_flares.max()

# set up figure
fig, ax = plt.subplots(figsize=(7, 5.5))

for color, group in mean_std.groupby("color"):
    # plt.hist(group.orbper_d, color=color, bins=bins)
    if color != "lightgrey":
        sns.kdeplot(data=group, x="number_of_flares",
                    fill=True, alpha=0.3, color=color)


# plt.xscale("log")
# plt.yscale("log")
plt.xlim(minper, maxper)
# plt.ylim(minper, maxper)
plt.xlabel("number of flares")
plt.ylabel("density")


In [ ]:
# define legend elements
legend = [Line2D([0], [0], marker='', color='black',
          markerfacecolor='black', markersize=10),
          Line2D([0], [0], marker='', color='blue',
          markerfacecolor='blue', markersize=10),
          Line2D([0], [0], marker='', color='green',
          markerfacecolor='green', markersize=10), ]

# define legend labels
labels = [r"$P_{rot} \geq 15$ d",
          r"$10 \leq P_{rot} < 15$ d",
          r"$P_{rot} < 10$ d", ]

# set up figure
fig, ax = plt.subplots(figsize=(7, 5.5))
bins = np.logspace(-2.1, 0, 10)
for color, group in mean_std.groupby("color"):
    if color != "lightgrey":
        # plt.hist(group["mean"], linewidth=4,
        #       color=color, bins=bins, alpha=0.5, label=color)
        sns.kdeplot(data=group, x="mean", fill=True, color=color, cut=0)


# plt.xscale("log")
# plt.yscale("log")
plt.xlim(0, 1)
# plt.ylim(minper, maxper)
#plt.xlabel("semi-major axis [AU]")
plt.xlabel("p-value of AD test")

# legeng
plt.legend(legend, labels, loc=2, fontsize=15)

# save figure
plt.savefig("../results/plots/2022_11_22_adtests_kde_vs_rotation.png", dpi=300)


In [ ]:


minper, maxper = mean_std.orbits_covered.min(), mean_std.orbits_covered.max()


# set up figure
fig, ax = plt.subplots(figsize=(7, 5.5))


for color, group in mean_std.groupby("color"):
    # print(rotp, np.mean(group.orbits_covered))
    # print(color)
    if color != "lightgrey":
        sns.kdeplot(data=group, x="orbits_covered", fill=True,
                    color=color, common_norm=True, alpha=.3, log_scale=True, cut=0
                    )

plt.xscale("log")
plt.yscale("log")
# plt.xlim(minper, maxper)
# plt.ylim(minper, maxper)
plt.ylabel("density")
plt.xlabel("number of orbits covered")

# legeng
plt.legend(legend, labels, loc=1, fontsize=13.5)


# Compare to fake p-value calculations

In [ ]:
# read in AD test results
adtests = pd.read_csv("../results/multistar_adtest.csv")

# get different modes
modes = adtests["period"].unique()

# read in the table of known parameters from the NASA Exoplanet Archive and TESS
sps_w_ad = pd.read_csv(
    "../results/params_of_star_planet_systems_with_AD_tests.csv")

# read in age and rotation period from the literature search
literature_params = pd.read_csv(
    "../results/2022_09_stellar_params_with_coherence_times.csv")
literature_params.TIC = literature_params.TIC.astype(str)

res = pd.DataFrame()
print(res.shape)
# aggregate the p-values
for mode in modes:

    mean_std = aggregate_pvalues(adtests, subsample="ED>1s", period=mode)
    print(mode)

    mean_std = mean_std[["TIC", "mean"]]

    if res.shape[0] == 0:
        res = mean_std.set_index("TIC")
    else:
        # join the the table to res as a new column
        res = res.merge(mean_std, on="TIC", how="left")

    res = res.rename(columns={"mean": f"mean_{mode}"})

# merge the AD test results with the stellar parameters
res = res.merge(literature_params, on=["TIC"], how="left")


In [ ]:
res.columns


In [ ]:
plt.figure(figsize=(9, 6.5))

for mode in modes:
    if mode == "orbit":
        o = 10
        histtype = "bar"
    else:
        o = 1
        histtype = "step"
    plt.hist(res[f"mean_{mode}"], bins=np.logspace(-3, 0, 15),
             label=mode.replace("_", " "), linewidth=3, zorder=o, alpha=.5, histtype=histtype)

plt.xscale("log")
plt.legend(loc=2)
plt.xlabel("p-value of AD test")
plt.ylabel("number of stars")
